In [11]:
import pandas as pd


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
data = pd.read_csv("/content/drive/MyDrive/week-6/data.csv")

In [38]:
data.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')

In [39]:
data.isna().sum()

,0
TransactionId,0
BatchId,0
AccountId,0
SubscriptionId,0
CustomerId,0
CurrencyCode,0
CountryCode,0
ProviderId,0
ProductId,0
ProductCategory,0


In [43]:
# Group by 'CustomerId'
grouped = data.groupby('CustomerId')

# Create aggregate features
data_aggregates = grouped.agg(
    TotalTransactionAmount=('Amount', 'sum'),
    AverageTransactionAmount=('Amount', 'mean'),
    TransactionCount=('TransactionId', 'count'),
    StdDevTransactionAmount=('Amount', 'std')
).reset_index()

# Join back to the original data (if needed)
data = pd.merge(data, data_aggregates, on='CustomerId', how='left')

In [41]:
# data = pd.get_dummies(data, columns=['ProductCategory', 'CountryCode', 'CurrencyCode'], drop_first=True)
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['ProductCategory'] = le.fit_transform(data['ProductCategory'])
data['CountryCode'] = le.fit_transform(data['CountryCode'])
data['CurrencyCode'] = le.fit_transform(data['CurrencyCode'])


In [44]:
data.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TotalTransactionAmount,AverageTransactionAmount,TransactionCount,StdDevTransactionAmount
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,0,0,ProviderId_6,ProductId_10,0,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0,109921.75,923.712185,119,3042.294251
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,0,0,ProviderId_4,ProductId_6,2,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0,109921.75,923.712185,119,3042.294251
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,0,0,ProviderId_6,ProductId_1,0,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0,1000.00,500.000000,2,0.000000
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,0,0,ProviderId_1,ProductId_21,8,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0,228727.20,6019.136842,38,17169.241610
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,0,0,ProviderId_4,ProductId_6,2,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0,228727.20,6019.136842,38,17169.241610


In [45]:
# Fill missing numerical values with mean
data['Amount'] = data['Amount'].fillna(data['Amount'].mean())

# Fill missing categorical values with mode
data['ProductCategory'] = data['ProductCategory'].fillna(data['ProductCategory'].mode()[0])


In [46]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
numerical_features = ['Amount', 'Value', 'TotalTransactionAmount', 'AverageTransactionAmount']
data[numerical_features] = scaler.fit_transform(data[numerical_features])


In [47]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data[numerical_features] = scaler.fit_transform(data[numerical_features])


In [48]:
!pip install xverse woe

In [52]:
!pip install --upgrade pandas


In [53]:
from xverse.transformer import WOE

woe = WOE()
data_woe = woe.fit_transform(data, data['FraudResult'])


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xverse/transformer/_binning.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bins_X_grouped = bins_X.groupby('Bins', as_index=True)
/usr/local/lib/python3.11/dist-packages/xverse/transformer/_binning.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bins_X_grouped = bins_X.groupby('Bins', as_index=True)


AttributeError: module 'pandas.core.algorithms' has no attribute 'quantile'